In [ ]:
import sys
import os
import numpy as np
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv("PATH_CUSUM"))
from source.model.incremental import RecursiveLeastSquares
from source.detector.cusum import CUSUM_Detector
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# set random seed for reproducibility
np.random.seed(42)

In [ ]:
n = 300

# Create time series components
t = np.arange(n)

# Trend component (slowly increasing)
trend = 0.05 * t

# Seasonal component (yearly pattern)
seasonal = 10 * np.sin(5 * np.pi * t / 365)

# Random noise
noise = np.random.normal(0, 5, n)

# Combine components
values = 50 + trend + seasonal + noise

# Abrupt shift on data

In [ ]:
values[150:] += 50  # Introduce a change point

# plot the time series
plt.figure(figsize=(15, 4))
plt.plot(t, values, label='Time Series Data')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Synthetic Time Series with Change Point')
plt.legend()
plt.show()

# Monitoring ml-model performance on data shift

In [ ]:
# Initialize model
num_lags = 5
model = RecursiveLeastSquares(num_variables=num_lags, 
                              forgetting_factor=0.999, 
                              initial_delta=1.0)

# Detect change points using CUSUM Detector
warmup_period = 50
cusum_detector = CUSUM_Detector(warmup_period=warmup_period, 
                                delta=1.5, 
                                threshold=4.0)


# Store predictions and observed values
list_predictions = []
list_observed = []
list_residuals = []

list_acc_changes = []
list_change_points = []
list_index_changes = []
list_output_detected = []

# Create a figure for the animation with grid layout
fig, axs = plt.subplots(3, 1, figsize=(20, 15))
prediction_ax, residual_ax, change_ax = axs
fig.tight_layout(pad=5.0)

# Animation function
def monitoring(i):

    global output_detected, list_change_points, list_output_detected, list_residuals
    
    # Ensure we don't go out of bounds
    if i >= len(values) - num_lags - 1:
        return
    
    # Get the input
    X = values[i:i + num_lags].reshape(-1, 1)

    # Make a prediction
    y_pred = model.predict(X)

    # Get the true value
    y = values[i + num_lags + 1].reshape(-1, 1)

    if i > 20:  # wait for residual to stabilize

        # Store prediction and observed values
        list_predictions.append(y_pred)
        list_observed.append(float(y[0][0]))

        # model params update
        model.update(X, y)

        # Compute residuals
        residuals = np.abs(y - y_pred)
        list_residuals.append(residuals[0][0])

        # Detect change points
        output_detected = cusum_detector.detection(residuals[0])
        list_acc_changes.append(float(output_detected[0][0]))
        list_change_points.append(output_detected[-1])
        list_output_detected.append(output_detected)

        # Clear the axes for new plots
        prediction_ax.clear()
        residual_ax.clear()
        change_ax.clear()

        # Plot predictions vs observed values
        prediction_ax.plot(list_observed, label='Observation', color='orange')
        prediction_ax.plot(list_predictions, label='Prediction', color='blue', marker='o', markersize=4)
        prediction_ax.fill_between(range(len(list_predictions)),
                                   list_predictions,
                                   list_observed,
                                   color='lightgray', 
                                   alpha=0.7)
        for i, is_cp in enumerate(list_change_points):
            if is_cp:
                prediction_ax.axvline(x=i, color='red', linestyle='--', alpha=0.5)
        prediction_ax.plot()
        prediction_ax.legend()
        prediction_ax.set_title(f'Predictions vs Observations')
        prediction_ax.set_xlabel('Time Step')
        prediction_ax.set_ylabel('Prediction Value')
        prediction_ax.grid()
        
        # Plot residuals
        residual_ax.plot(list_residuals, label='Residuals', color='red', marker='o', markersize=4) 
        residual_ax.fill_between(range(len(list_residuals)),
                                 0,
                                 list_residuals,
                                 color='red', 
                                 alpha=0.1)                     
        residual_ax.legend()
        residual_ax.set_title('Residuals')
        residual_ax.set_xlabel('Time Step')
        residual_ax.set_ylabel('Residual Value')
        residual_ax.grid()

        # Plot detected change points
        change_ax.plot(list_acc_changes, label='Cumulative Change', color='green', marker='o', markersize=4)
        change_ax.fill_between(range(len(list_acc_changes)),
                                0,
                                list_acc_changes,
                                color='green', 
                                alpha=0.5)
        change_ax.axhline(y=cusum_detector.threshold, color='r', linestyle='--', label='CUSUM Threshold')
        change_ax.legend()
        change_ax.set_title('Detected Change Points')
        change_ax.set_xlabel('Time Step')
        change_ax.set_ylabel('Cumulative Change Value')
        change_ax.grid()

# Create the animation
ani = FuncAnimation(fig, monitoring, frames=len(values) - num_lags - 1, repeat=False)  #

# Save the animation
ani.save('monitoring.gif', writer='pillow', fps=100)
plt.close(fig)  # Prevent the figure from displaying

In [ ]:
import IPython.display as display

# Display the saved GIF
display.Image(filename='monitoring.gif')